# Tensors and operations

In [1]:
import tensorflow as tf 
t = tf.constant([[1.,2.,3.], [4.,5.,6.]]) # matrix
t

2025-11-05 06:42:43.815270: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-05 06:42:48.035784: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-05 06:43:19.292683: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
E0000 00:00:1762325024.091347     445 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1762325024.101129     445 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are install

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [2]:
print(t.shape)
print(t.dtype)

(2, 3)
<dtype: 'float32'>


In [3]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [4]:
t[...,1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [5]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [6]:
t @ tf.transpose(t)  # matrix multiplication

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

# tensors and numpy

In [7]:
import numpy as np
a = np.array([2., 4.,5])

tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [8]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [9]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [10]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

# customizing models and training algorithms

In [12]:
# custom loss function with threshold
def create_huber(threshold):
    def huber_loss(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold **2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_loss

# model.compile(loss=create_huber(2.0), optimizer='nadam')

but, when you save the model the threshold will not be saved. this can be fixed by creating a subclass of the tf.keras.losses.Loss class

In [13]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold **2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}
    
'''
compiling:
   model.compile(loss=HuberLoss(2.), optimizer='nadam')

saving will save threshold along with it
   model = tf.keras.models.load_model("my_model", custom_objects={"HuberLoss": HuberLoss})
'''

'\ncompiling:\n   model.compile(loss=HuberLoss(2.), optimizer=\'nadam\')\n\nsaving will save threshold along with it\n   model = tf.keras.models.load_model("my_model", custom_objects={"HuberLoss": HuberLoss})\n'